In [15]:
import numpy as np
import open3d as o3d
import os
import torch

# Define paths
data_path = "/home/yasin/Downloads/road/semantickitti_small/sequences/00/velodyne/000390.bin"  # Example path to LiDAR data
label_path = "/home/yasin/Downloads/road/semantickitti_small/sequences/00/labels/000390.label"  # Example path to label file

# SemanticKITTI color map for each label (20 classes in SemanticKITTI)
# Color each class with RGB values for visualization
sem2sem = {
    0 : 0,     # "unlabeled"
    1 : 0,     # "outlier" mapped to "unlabeled" --------------------------mapped
    10: 1,     # "car"
    11: 2,     # "bicycle"
    13: 5,     # "bus" mapped to "other-vehicle" --------------------------mapped
    15: 3,     # "motorcycle"
    16: 5,     # "on-rails" mapped to "other-vehicle" ---------------------mapped
    18: 4,     # "truck"
    20: 5,     # "other-vehicle"
    30: 6,     # "person"
    31: 7,     # "bicyclist"
    32: 8,     # "motorcyclist"
    40: 9,     # "road"
    44: 10,    # "parking"
    48: 11,    # "sidewalk"
    49: 12,    # "other-ground"
    50: 13,    # "building"
    51: 14,    # "fence"
    52: 0,     # "other-structure" mapped to "unlabeled" ------------------mapped
    60: 9,     # "lane-marking" to "road" ---------------------------------mapped
    70: 15,    # "vegetation"
    71: 16,    # "trunk"
    72: 17,    # "terrain"
    80: 18,    # "pole"
    81: 19,    # "traffic-sign"
    99: 0,     # "other-object" to "unlabeled" ----------------------------mapped
    252: 1,    # "moving-car" to "car" ------------------------------------mapped
    253: 7,    # "moving-bicyclist" to "bicyclist" ------------------------mapped
    254: 6,    # "moving-person" to "person" ------------------------------mapped
    255: 8,    # "moving-motorcyclist" to "motorcyclist" ------------------mapped
    256: 5,    # "moving-on-rails" mapped to "other-vehicle" --------------mapped
    257: 5,    # "moving-bus" mapped to "other-vehicle" -------------------mapped
    258: 4,    # "moving-truck" to "truck" --------------------------------mapped
    259: 5     # "moving-other-vehicle" to "other-vehicle" ----------------mapped
}

color_map = {
    0: [0, 0, 0],          # "unlabeled" - black
    1: [245, 150, 100],    # "car" - red
    2: [245, 230, 100],    # "bicycle" - yellow
    3: [250, 80, 100],     # "motorcycle" - pink
    4: [150, 60, 30],      # "truck" - brown
    5: [180, 30, 80],      # "other-vehicle" - dark red
    6: [255, 0, 0],        # "person" - bright red
    7: [30, 30, 255],      # "bicyclist" - blue
    8: [200, 40, 255],     # "motorcyclist" - purple
    9: [90, 30, 150],      # "road" - dark purple
    10: [255, 0, 255],     # "parking" - magenta
    11: [255, 150, 255],   # "sidewalk" - light pink
    12: [75, 0, 75],       # "other-ground" - dark purple
    13: [75, 0, 175],      # "building" - dark blue
    14: [0, 200, 255],     # "fence" - sky blue
    15: [50, 120, 255],    # "vegetation" - greenish blue
    16: [0, 175, 0],       # "trunk" - green
    17: [0, 60, 135],      # "terrain" - dark cyan
    18: [80, 240, 150],    # "pole" - light green
    19: [150, 240, 255],   # "traffic-sign" - light cyan
    20: [150, 240, 0],   # "traffic-sign" - light cyan
    21: [150, 0, 255],   # "traffic-sign" - light cyan
    22: [0, 240, 255]   # "traffic-sign" - light cyan
}


color_map = torch.tensor(list(color_map.values()))

# Create lookup tensor for class mapping
mapdict = sem2sem
max_index = max(mapdict.keys())
lookup_tensor = torch.zeros(max_index + 1, dtype=torch.long)
for old_idx, new_idx in mapdict.items():
    lookup_tensor[old_idx] = new_idx

# Load point cloud and label data
def load_point_cloud(file_path):
    """Load point cloud from a .bin file."""
    point_cloud = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    return point_cloud

def load_labels(label_path):
    """Load labels from a .label file."""
    labels = np.fromfile(label_path, dtype=np.int32) & 0xFFFF
    labels = lookup_tensor[labels]
    return labels

def colorize_point_cloud(points, labels, color_map):
    """Apply colors to point cloud based on labels and a predefined color map."""
    colors = np.array([color_map[label] for label in labels]) / 255.0
    return colors

# Load data
points_all = load_point_cloud(data_path)
points = points_all[:, :3]  # Only XYZ, ignore intensity
labels = load_labels(label_path)
colors = colorize_point_cloud(points, labels, color_map)

# Create Open3D point cloud object and set points and colors
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Add coordinate frame for visualization of the coordinate system
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0, origin=[0, 0, 0])

# Visualize the point cloud with the coordinate frame
o3d.visualization.draw_geometries([pcd, coordinate_frame], window_name="SemanticKITTI Point Cloud", width=800, height=600)

In [16]:
from glob import glob
wifiles = sorted(glob('/home/yasin/Downloads/road/waymodataset/training/images/*'))
wlfiles = sorted(glob('/home/yasin/Downloads/road/waymodataset/training/labels/*'))

root = '/home/yasin/Downloads/road/waymodataset/training'
calibroot = '/home/yasin/Downloads/road/waymodataset/training/calibs'
images = sorted(glob(f'{root}/images/*.npz'))
labels = sorted(glob(f'{root}/labels/*.npz'))
calibs = sorted(glob(f'{calibroot}/*.csv'))
image_to_calibration = dict()
for image_file in images:
    for calibration_file in calibs:
        prefix = calibration_file.split('.')[0].split('/')[-1]
        if image_file.split('/')[-1].startswith(prefix):
            image_to_calibration[image_file] = calibration_file
            break

wcfiles = [image_to_calibration[i] for i in wifiles]

In [17]:
# Load data
index = 200
sx = 1.43
sz = 2.184
sy = 0
s = np.array([sx,sy,sz])
waymo_points_all = np.load(wifiles[index])['array'].astype(np.float32)
waymo_points = waymo_points_all[1:4,...]
waymo_points[2:4] = -1*waymo_points[2:4] 
waymo_points[0] = waymo_points[0]-sx
waymo_points[2] = waymo_points[2]+sz
waymo_points = np.transpose(waymo_points,(1,2,0)).reshape(-1,3)
waymo_labels = np.load(wlfiles[index])['array'].reshape(-1)
waymo_points = waymo_points[(waymo_labels == 1) | (waymo_labels == 18)| (waymo_labels == 0)]
waymo_labels = waymo_labels[(waymo_labels == 1) | (waymo_labels == 18)| (waymo_labels == 0)]
waymo_colors = colorize_point_cloud(waymo_points, waymo_labels, color_map)


# Create Open3D point cloud object and set points and colors
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(waymo_points)
pcd.colors = o3d.utility.Vector3dVector(waymo_colors)

# Add coordinate frame for visualization of the coordinate system
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0, origin=[0, 0, 0])

# Visualize the point cloud with the coordinate frame
o3d.visualization.draw_geometries([pcd, coordinate_frame], window_name="SemanticKITTI Point Cloud", width=800, height=600)

In [18]:
import pandas as pd
keytr = '[LiDARCalibrationComponent].extrinsic.transform'
keyinc = '[LiDARCalibrationComponent].beam_inclination.values'

In [19]:
df = pd.read_csv(wcfiles[0])
arstr = df[keytr].values[0]
array = np.fromstring(arstr.strip('[]'), sep=' ')

In [ ]:
s = list(set(wcfiles))

In [ ]:
for index in range(len(s)):
    df = pd.read_csv(s[index])
    arstr = df[keyinc].values[0]
    array = np.fromstring(arstr.strip('[]'), sep=' ')
    print(array)

In [31]:
def range_image_to_point_cloud_with_labels(range_image, inclination_angles, labels):
    """
    Converts a range image to a point cloud and includes corresponding labels.

    Parameters:
    - range_image: A 2D numpy array of shape (64, 2650), representing the range values for each pixel.
    - inclination_angles: A 1D numpy array of shape (64,), representing the inclination angle for each row.
    - labels: A 2D numpy array of shape (64, 2650), representing the label for each pixel.

    Returns:
    - point_cloud: A 2D numpy array of shape (64 * 2650, 3) containing the (x, y, z) coordinates of the point cloud.
    - point_labels: A 1D numpy array of shape (64 * 2650,) containing the labels associated with each point.
    """

    # Create a meshgrid for azimuth angles for each pixel column
    azimuth_angles = np.linspace(-np.pi, np.pi, range_image.shape[1])

    # Repeat the inclination and azimuth angles to match the range image shape
    inclination = np.repeat(inclination_angles[:, np.newaxis], range_image.shape[1], axis=1)
    azimuth = np.tile(azimuth_angles, (range_image.shape[0], 1))

    # Calculate the Cartesian coordinates
    x = range_image * np.cos(inclination) * np.cos(azimuth)
    y = range_image * np.cos(inclination) * np.sin(azimuth)
    z = range_image * np.sin(inclination)

    # Stack the x, y, z coordinates to form the point cloud
    point_cloud = np.stack((x, y, z), axis=-1).reshape(-1, 3)

    # Flatten the labels array to match the shape of the point cloud
    point_labels = labels.flatten()

    return point_cloud, point_labels

def get_inc(filename):
    df = pd.read_csv(filename)
    arstr = df[keyinc].values[0]
    array = np.fromstring(arstr.strip('[]'), sep=' ')
    return array

In [38]:
index = 200
rng = np.load(wifiles[index])['array'].astype(np.float32)[0]
lbl = np.load(wlfiles[index])['array']
clb = get_inc(wcfiles[index])
clb = newinc
point_cloud, point_labels = range_image_to_point_cloud_with_labels(rng, clb, lbl)

In [36]:
clb = get_inc(wcfiles[index])
point_cloud, point_labels = range_image_to_point_cloud_with_labels(rng, clb, lbl)

In [26]:
df = pd.read_csv(wcfiles[0])
arstr = df[keyinc].values[0]
array = np.fromstring(arstr.strip('[]'), sep=' ')
mm = np.array([-0.313889,	0.040506])

TypeError: 'float' object cannot be interpreted as an integer

In [27]:
step = (0.313889+0.040506)/63.0
newinc = [i*step+-0.313889 for i in range(64)]
newinc = np.array(newinc)

In [39]:
waymo_points = point_cloud
waymo_labels = point_labels
waymo_colors = colorize_point_cloud(waymo_points, waymo_labels, color_map)

# Create Open3D point cloud object and set points and colors
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(waymo_points)
pcd.colors = o3d.utility.Vector3dVector(waymo_colors)

# Add coordinate frame for visualization of the coordinate system
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0, origin=[0, 0, 0])

# Visualize the point cloud with the coordinate frame
o3d.visualization.draw_geometries([pcd, coordinate_frame], window_name="SemanticKITTI Point Cloud", width=800, height=600)